# The purpose of this notebook is to write code which will give us a sense of how a stock will move due to earnings based on historical data and performance

## We'll start with simple EDA and rule-based type stuff, and then potentially move onto ML models

### First started around 4/19/20

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import yfinance as yf

In [2]:
dpz = yf.Ticker('DPZ') # get Domino's Pizza
df = dpz.history(period='max') # get as much data as possible
df = df.drop(columns=['Dividends', 'Stock Splits']) # we don't care about dividends or stock splits for this
df.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2021-07-19,483.350006,484.690002,473.809998,478.489990,578600
2021-07-20,475.829987,478.549988,471.279999,474.970001,593000
2021-07-21,475.489990,476.450012,465.929993,470.369995,614700
2021-07-22,484.989990,548.719971,484.000000,538.820007,3038100
2021-07-23,541.989990,542.520020,519.469971,525.450012,1342100


In [3]:
df.Close.plot()
plt.show()

## Earnings:

In [4]:
# load earnings data
earnings = pd.read_csv('../data/dpz_earnings.csv')
earnings.head()

,ticker,company,earnings_date,eps_estimate,reported_eps,surprise_percent
0,DPZ,Domino's Pizza Inc,Apr 23 2020 4,2.32,NaN,NaN
1,DPZ,Domino's Pizza Inc,Feb 20 2020 12,2.98,3.13,5.03
2,DPZ,Domino's Pizza Inc,Oct 08 2019 11,2.07,2.05,-0.97
3,DPZ,Domino's Pizza Inc,Jul 16 2019 11,2.02,2.19,8.42
4,DPZ,Domino's Pizza Inc,Apr 24 2019 11,2.09,2.20,5.26


In [5]:
# convert earnings_date to datetime object
earnings.earnings_date = earnings.earnings_date.apply(lambda x: datetime.strptime(x, '%b %d %Y %H'))
earnings.earnings_date = earnings.earnings_date.apply(lambda x: pd.to_datetime(x).floor('D'))
earnings.head()

,ticker,company,earnings_date,eps_estimate,reported_eps,surprise_percent
0,DPZ,Domino's Pizza Inc,2020-04-23,2.32,NaN,NaN
1,DPZ,Domino's Pizza Inc,2020-02-20,2.98,3.13,5.03
2,DPZ,Domino's Pizza Inc,2019-10-08,2.07,2.05,-0.97
3,DPZ,Domino's Pizza Inc,2019-07-16,2.02,2.19,8.42
4,DPZ,Domino's Pizza Inc,2019-04-24,2.09,2.20,5.26


In [6]:
earnings.earnings_date.min(), earnings.earnings_date.max()

(Timestamp('2017-02-28 00:00:00'), Timestamp('2020-04-23 00:00:00'))

In [7]:
earnings = earnings.dropna(how='any')
earnings = earnings.reset_index(drop=True)
earnings

,ticker,company,earnings_date,eps_estimate,reported_eps,surprise_percent
0,DPZ,Domino's Pizza Inc,2020-02-20,2.98,3.13,5.03
1,DPZ,Domino's Pizza Inc,2019-10-08,2.07,2.05,-0.97
2,DPZ,Domino's Pizza Inc,2019-07-16,2.02,2.19,8.42
3,DPZ,Domino's Pizza Inc,2019-04-24,2.09,2.20,5.26
4,DPZ,Domino's Pizza Inc,2019-02-21,2.69,2.62,-2.60
5,DPZ,Domino's Pizza Inc,2018-10-16,1.75,1.95,11.43
6,DPZ,Domino's Pizza Inc,2018-07-19,1.75,1.84,5.14
7,DPZ,Domino's Pizza Inc,2018-04-26,1.77,2.00,12.99
8,DPZ,Domino's Pizza Inc,2018-02-20,1.95,2.09,7.18
9,DPZ,Domino's Pizza Inc,2017-10-12,1.23,1.27,3.25


In [8]:
earnings['beat'] = earnings.apply(lambda x: 1 if x['reported_eps'] > x['eps_estimate'] else 0, axis=1)
earnings

,ticker,company,earnings_date,eps_estimate,reported_eps,surprise_percent,beat
0,DPZ,Domino's Pizza Inc,2020-02-20,2.98,3.13,5.03,1
1,DPZ,Domino's Pizza Inc,2019-10-08,2.07,2.05,-0.97,0
2,DPZ,Domino's Pizza Inc,2019-07-16,2.02,2.19,8.42,1
3,DPZ,Domino's Pizza Inc,2019-04-24,2.09,2.20,5.26,1
4,DPZ,Domino's Pizza Inc,2019-02-21,2.69,2.62,-2.60,0
5,DPZ,Domino's Pizza Inc,2018-10-16,1.75,1.95,11.43,1
6,DPZ,Domino's Pizza Inc,2018-07-19,1.75,1.84,5.14,1
7,DPZ,Domino's Pizza Inc,2018-04-26,1.77,2.00,12.99,1
8,DPZ,Domino's Pizza Inc,2018-02-20,1.95,2.09,7.18,1
9,DPZ,Domino's Pizza Inc,2017-10-12,1.23,1.27,3.25,1


In [9]:
percent_beat = np.sum(earnings.beat)/len(earnings)*100
percent_beat

84.61538461538461

###  We see that over the last ~3 years, or 13 Quarters, Domino's has beat earnings estimates 84.62% of times

# What kind of suprise do we tend to see on average? What if we split by beat vs not-beat?

# What kind of move do we see following earnings announcement?

In [10]:
dates = earnings.earnings_date.values
#dates = [date for date in dates]
dates

array(['2020-02-20T00:00:00.000000000', '2019-10-08T00:00:00.000000000',
       '2019-07-16T00:00:00.000000000', '2019-04-24T00:00:00.000000000',
       '2019-02-21T00:00:00.000000000', '2018-10-16T00:00:00.000000000',
       '2018-07-19T00:00:00.000000000', '2018-04-26T00:00:00.000000000',
       '2018-02-20T00:00:00.000000000', '2017-10-12T00:00:00.000000000',
       '2017-07-25T00:00:00.000000000', '2017-04-27T00:00:00.000000000',
       '2017-02-28T00:00:00.000000000'], dtype='datetime64[ns]')

In [11]:
beats = earnings.beat.values

In [12]:
df = df[df.index.get_level_values('Date') >= dates.min() - pd.to_timedelta('30 days')]
df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2017-01-30,164.223942,166.997314,164.089592,166.910934,655200
2017-01-31,166.690195,168.024099,165.548216,167.496292,822000
2017-02-01,167.755379,168.743811,163.197070,163.993576,662200
2017-02-02,164.742142,167.976138,163.772906,167.851395,1154800
2017-02-03,168.158410,170.144880,167.285145,168.791779,749900


In [13]:
fig,ax = plt.subplots(figsize=(15,8))
df.Close.plot()
for date,beat in zip(dates,beats):
    if beat == 1:
        plt.axvline(x=date, c='g', alpha=0.5, label='beat')
    else:
        plt.axvline(x=date, c='r', alpha=0.5, label='miss')
plt.legend()
plt.grid()
plt.show()

In [14]:
dates = [date for date in dates]

In [15]:
df = df.reset_index()
df['earnings'] = df.Date.apply(lambda x: 1 if x in dates else 0)
df = df.set_index('Date')
df.head()

,Open,High,Low,Close,Volume,earnings
Date,,,,,,
2017-01-30,164.223942,166.997314,164.089592,166.910934,655200,0
2017-01-31,166.690195,168.024099,165.548216,167.496292,822000,0
2017-02-01,167.755379,168.743811,163.197070,163.993576,662200,0
2017-02-02,164.742142,167.976138,163.772906,167.851395,1154800,0
2017-02-03,168.158410,170.144880,167.285145,168.791779,749900,0


In [16]:
df['day_gain'] = df.Close - df.Open
df.head()

,Open,High,Low,Close,Volume,earnings,day_gain
Date,,,,,,,
2017-01-30,164.223942,166.997314,164.089592,166.910934,655200,0,2.686992
2017-01-31,166.690195,168.024099,165.548216,167.496292,822000,0,0.806098
2017-02-01,167.755379,168.743811,163.197070,163.993576,662200,0,-3.761803
2017-02-02,164.742142,167.976138,163.772906,167.851395,1154800,0,3.109253
2017-02-03,168.158410,170.144880,167.285145,168.791779,749900,0,0.633369


In [17]:
df['prev_close'] = df.Close.shift()
df.head()

,Open,High,Low,Close,Volume,earnings,day_gain,prev_close
Date,,,,,,,,
2017-01-30,164.223942,166.997314,164.089592,166.910934,655200,0,2.686992,NaN
2017-01-31,166.690195,168.024099,165.548216,167.496292,822000,0,0.806098,166.910934
2017-02-01,167.755379,168.743811,163.197070,163.993576,662200,0,-3.761803,167.496292
2017-02-02,164.742142,167.976138,163.772906,167.851395,1154800,0,3.109253,163.993576
2017-02-03,168.158410,170.144880,167.285145,168.791779,749900,0,0.633369,167.851395


In [18]:
df['prev_close_to_open'] = df.Open - df.prev_close
df['prev_close_to_close'] = df.Close - df.prev_close
df.head()

,Open,High,Low,Close,Volume,earnings,day_gain,prev_close,prev_close_to_open,prev_close_to_close
Date,,,,,,,,,,
2017-01-30,164.223942,166.997314,164.089592,166.910934,655200,0,2.686992,NaN,NaN,NaN
2017-01-31,166.690195,168.024099,165.548216,167.496292,822000,0,0.806098,166.910934,-0.220740,0.585358
2017-02-01,167.755379,168.743811,163.197070,163.993576,662200,0,-3.761803,167.496292,0.259087,-3.502716
2017-02-02,164.742142,167.976138,163.772906,167.851395,1154800,0,3.109253,163.993576,0.748566,3.857819
2017-02-03,168.158410,170.144880,167.285145,168.791779,749900,0,0.633369,167.851395,0.307015,0.940384


In [19]:
df['open_pct_change'] = df.apply(lambda x: (x.prev_close_to_open/x.prev_close)*100, axis=1)
df['close_pct_change'] = df.apply(lambda x: (x.prev_close_to_close/x.prev_close)*100, axis=1)

In [20]:
res_df = df[df.earnings == 1]
res_df['beat']= beats[::-1]
res_df

<ipython-input-20-ea7320c8c529>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_df['beat']= beats[::-1]


,Open,High,Low,Close,Volume,earnings,day_gain,prev_close,prev_close_to_open,prev_close_to_close,open_pct_change,close_pct_change,beat
Date,,,,,,,,,,,,,
2017-02-28,179.040817,184.088540,174.904739,182.150055,2158200,1,3.109238,177.937225,1.103592,4.212830,0.620214,2.367593,1
2017-04-27,179.992834,181.676358,176.789325,178.501709,2422000,1,-1.491125,174.086075,5.906759,4.415634,3.393011,2.536466,1
2017-07-25,199.586769,199.586769,185.181808,185.509628,4249500,1,-14.077140,206.307220,-6.720452,-20.797592,-3.257497,-10.080884,1
2017-10-12,193.260801,198.489817,190.119528,194.304672,4872900,1,1.043871,202.240021,-8.979220,-7.935349,-4.439883,-3.923728,1
2018-02-20,208.822701,220.063280,201.768274,214.627106,4770100,1,5.804404,213.900360,-5.077659,0.726746,-2.373843,0.339759,1
2018-04-26,242.406563,247.700553,238.958180,243.436218,1709100,1,1.029655,226.845139,15.561425,16.591080,6.859933,7.313835,1
2018-07-19,267.666177,277.175615,265.622174,269.369507,1970200,1,1.703330,276.114716,-8.448539,-6.745209,-3.059793,-2.442901,1
2018-10-16,258.236088,261.015448,250.824502,253.194260,2390800,1,-5.041829,266.174377,-7.938289,-12.980118,-2.982364,-4.876547,1
2019-02-21,255.561378,256.460537,244.360770,247.283081,3877500,1,-8.278297,272.186310,-16.624932,-24.903229,-6.107924,-9.149332,0


In [21]:
res_df.open_pct_change.median(), res_df.open_pct_change.mean()

(-2.982364129390703, 0.4694236898515338)

In [22]:
res_df.close_pct_change.median(), res_df.close_pct_change.mean()

(0.33975894435323745, 0.6591456910693417)

In [23]:
res_df[-5:].open_pct_change.mean()

2.268546022913722

In [24]:
temp = earnings[['earnings_date', 'surprise_percent']]
#temp

In [25]:
res_df = res_df.merge(temp, left_on='Date', right_on='earnings_date')
res_df = res_df.set_index(['earnings_date'])
res_df.index.name = 'Date'
res_df

,Open,High,Low,Close,Volume,earnings,day_gain,prev_close,prev_close_to_open,prev_close_to_close,open_pct_change,close_pct_change,beat,surprise_percent
Date,,,,,,,,,,,,,,
2017-02-28,179.040817,184.088540,174.904739,182.150055,2158200,1,3.109238,177.937225,1.103592,4.212830,0.620214,2.367593,1,2.78
2017-04-27,179.992834,181.676358,176.789325,178.501709,2422000,1,-1.491125,174.086075,5.906759,4.415634,3.393011,2.536466,1,8.62
2017-07-25,199.586769,199.586769,185.181808,185.509628,4249500,1,-14.077140,206.307220,-6.720452,-20.797592,-3.257497,-10.080884,1,7.32
2017-10-12,193.260801,198.489817,190.119528,194.304672,4872900,1,1.043871,202.240021,-8.979220,-7.935349,-4.439883,-3.923728,1,3.25
2018-02-20,208.822701,220.063280,201.768274,214.627106,4770100,1,5.804404,213.900360,-5.077659,0.726746,-2.373843,0.339759,1,7.18
2018-04-26,242.406563,247.700553,238.958180,243.436218,1709100,1,1.029655,226.845139,15.561425,16.591080,6.859933,7.313835,1,12.99
2018-07-19,267.666177,277.175615,265.622174,269.369507,1970200,1,1.703330,276.114716,-8.448539,-6.745209,-3.059793,-2.442901,1,5.14
2018-10-16,258.236088,261.015448,250.824502,253.194260,2390800,1,-5.041829,266.174377,-7.938289,-12.980118,-2.982364,-4.876547,1,11.43
2019-02-21,255.561378,256.460537,244.360770,247.283081,3877500,1,-8.278297,272.186310,-16.624932,-24.903229,-6.107924,-9.149332,0,-2.60


In [26]:
import seaborn as sns
sns.regplot(x='surprise_percent', y='open_pct_change', data=res_df)
plt.axhline(y=0, c='r')
plt.show()

# What is the typical run-up to earnings?